In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Load data (adjust path if needed)
df = pd.read_csv("data/data.csv", sep=';')

# Check
df.head()


,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [1]:
import pandas as pd
import numpy as np

# Load original dataset 
df = pd.read_csv("data/data.csv", sep=";")

# Select the 12 variables needed for the visualizations
cols = [
    "Debtor",
    "Tuition fees up to date",
    "Scholarship holder",
    "Admission grade",
    "Previous qualification (grade)",
    "Curricular units 1st sem (enrolled)",
    "Curricular units 1st sem (evaluations)",
    "Curricular units 1st sem (approved)",
    "Curricular units 1st sem (grade)",
    "Mother's qualification",
    "Father's qualification",
    "Target"
]
df = df[cols].copy()

# ---- Clean missing values for grade columns only (NA = missing) ----
grade_cols = ["Admission grade", "Previous qualification (grade)"]
df[grade_cols] = df[grade_cols].replace({0: np.nan})

# ---- Convert financial stability fields to Yes/No ----
binary_map = {0: "No", 1: "Yes"}
df["Debtor"] = df["Debtor"].map(binary_map)
df["Tuition fees up to date"] = df["Tuition fees up to date"].map(binary_map)
df["Scholarship holder"] = df["Scholarship holder"].map(binary_map)

# ---- Clean Target variable (dropout status) ----
df["Target"] = df["Target"].astype(str).str.title()

# ---- Parental qualification mapping----
qualification_map = {
    1:  "Secondary Education - 12th Year",
    2:  "Bachelor's Degree",
    3:  "Higher Education - Degree",
    4:  "Master's Degree",
    5:  "Doctorate",
    6:  "Incomplete Higher Education",
    9:  "12th Year - Not Completed",
    10: "11th Year - Not Completed",
    11: "7th Year (Old Program)",
    12: "11th Year - Other",
    14: "10th Year of Schooling",
    18: "General Commerce Course",
    19: "Basic Education - 3rd Cycle",
    22: "Technical-Professional Course",
    26: "7th Year of Schooling",
    27: "General High School - 2nd Cycle",
    29: "9th Year - Not Completed",
    30: "8th Year of Schooling",
    34: "Unknown",
    35: "Cannot Read or Write",
    36: "Can Read (No 4th Year)",
    37: "Basic Education - 1st Cycle (4th/5th)",
    38: "Basic Education - 2nd Cycle (6th/7th/8th)",
    39: "Technological Specialization Course",
    40: "Higher Education - Degree (1st cycle)",
    41: "Specialized Higher Studies Course",
    42: "Professional Higher Technical Course",
    43: "Master's - 2nd cycle",
    44: "Doctorate - 3rd cycle"
}

# keep numeric codes as-is for plotting
df["Mother_qual_code"] = df["Mother's qualification"]
df["Father_qual_code"] = df["Father's qualification"]

# Add readable labels for hover text / tables
df["Mother_qual_label"] = df["Mother's qualification"].map(qualification_map)
df["Father_qual_label"] = df["Father's qualification"].map(qualification_map)

# ---- Save cleaned dataset ----
df.to_csv("data/cleaned_students.csv", index=False)
print("Cleaning completed, clean file saved as data/cleaned_students.csv")


Cleaning completed, clean file saved as data/cleaned_students.csv


In [9]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio

df = pd.read_csv("data/cleaned_students.csv")

font_family = "Georgia, 'Times New Roman', serif"

# ------------------------------------------------------
# Qualification Label Map
# ------------------------------------------------------
qualification_map = {
    1:  "Secondary Education - 12th Year",
    2:  "Bachelor's Degree",
    3:  "Higher Education - Degree",
    4:  "Master's Degree",
    5:  "Doctorate",
    6:  "Incomplete Higher Education",
    9:  "12th Year - Not Completed",
    10: "11th Year - Not Completed",
    11: "7th Year (Old Program)",
    12: "11th Year - Other",
    14: "10th Year of Schooling",
    18: "General Commerce Course",
    19: "Basic Education - 3rd Cycle",
    22: "Technical-Professional Course",
    26: "7th Year of Schooling",
    27: "General High School - 2nd Cycle",
    29: "9th Year - Not Completed",
    30: "8th Year of Schooling",
    34: "Unknown",
    35: "Cannot Read or Write",
    36: "Can Read (No 4th Year)",
    37: "Basic Education - 1st Cycle (4th/5th)",
    38: "Basic Education - 2nd Cycle (6th/7th/8th)",
    39: "Technological Specialization Course",
    40: "Higher Education - Degree (1st cycle)",
    41: "Specialized Higher Studies Course",
    42: "Professional Higher Technical Course",
    43: "Master's - 2nd cycle",
    44: "Doctorate - 3rd cycle"
}

# ------------------------------------------------------
# Qualification Rank Map
# ------------------------------------------------------
qual_rank_map = {
    35: 0, 36: 1,
    37: 2, 26: 3, 11: 3,
    38: 4, 30: 5, 29: 6, 19: 7,
    14: 8, 10: 9, 12: 10, 18: 11,
    27: 12, 9: 13, 1: 14,
    22: 15, 39: 16,
    42: 17, 40: 18, 3: 19,
    2: 20, 41: 21,
    4: 22, 43: 23,
    5: 24, 44: 25,
    34: 12
}

# ------------------------------------------------------
# Data Processing
# ------------------------------------------------------
df["Mother_qual_label"] = df["Mother's qualification"].map(qualification_map)
df["Father_qual_label"] = df["Father's qualification"].map(qualification_map)

df["Mother_qual_rank"] = df["Mother's qualification"].map(qual_rank_map)
df["Father_qual_rank"]  = df["Father's qualification"].map(qual_rank_map)

df = df.dropna(subset=["Mother_qual_rank", "Father_qual_rank"])

df["SES_score"] = (df["Mother_qual_rank"] + df["Father_qual_rank"]) / 2
q1, q2 = df["SES_score"].quantile([0.33, 0.66])

ses_groups = {
    "All SES": df,
    "Low SES": df[df["SES_score"] <= q1],
    "Mid SES": df[(df["SES_score"] > q1) & (df["SES_score"] < q2)],
    "High SES": df[df["SES_score"] >= q2]
}

ses_labels = list(ses_groups.keys())

fig = go.Figure()

# ------------------------------------------------------
# Add traces
# ------------------------------------------------------
for i, label in enumerate(ses_labels):

    sub = ses_groups[label]

    # Heatmap
    fig.add_trace(go.Histogram2d(
        x=sub["Mother_qual_rank"],
        y=sub["Father_qual_rank"],
        colorscale="Blues",
        zsmooth="best",
        nbinsx=25, nbinsy=25,
        opacity=0.65,
        name=f"{label} Density",
        showscale=(label == "All SES")
    ))

    # Dropouts
    drop = sub[sub["Target"] == "Dropout"]
    fig.add_trace(go.Scatter(
        x=drop["Mother_qual_rank"],
        y=drop["Father_qual_rank"],
        mode="markers",
        marker=dict(color="red", size=7, opacity=0.85),
        name="Dropout",
        visible=(label == "All SES")
    ))

    # Graduates
    grad = sub[sub["Target"] == "Graduate"]
    fig.add_trace(go.Scatter(
        x=grad["Mother_qual_rank"],
        y=grad["Father_qual_rank"],
        mode="markers",
        marker=dict(color="green", size=7, opacity=0.8),
        name="Graduate",
        visible=False
    ))

    # Enrolled
    enr = sub[sub["Target"] == "Enrolled"]
    fig.add_trace(go.Scatter(
        x=enr["Mother_qual_rank"],
        y=enr["Father_qual_rank"],
        mode="markers",
        marker=dict(color="orange", size=7, opacity=0.8),
        name="Enrolled",
        visible=False
    ))

# ------------------------------------------------------
# Dropdown Logic
# ------------------------------------------------------
traces_per_group = 4
total = traces_per_group * len(ses_labels)

buttons = []
for i, label in enumerate(ses_labels):
    vis = [False] * total
    start, end = i * traces_per_group, (i + 1) * traces_per_group
    for j in range(start, end):
        vis[j] = True
    buttons.append(dict(
        label=label,
        method="update",
        args=[{"visible": vis},
              {"title": f"Socioeconomic Status Landscape ({label})"}]
    ))

# ------------------------------------------------------
# Layout and Font Settings
# ------------------------------------------------------
fig.update_layout(
    title=dict(
        text="Socioeconomic Status Landscape (All SES)",
        font=dict(family=font_family, size=22, color="#101010")
    ),
    font=dict(family=font_family, size=14, color="#333"),
    hoverlabel=dict(font=dict(family=font_family, size=13)),
    width=1400,
    height=650,
    margin=dict(l=70, r=150, t=120, b=70),
    legend=dict(
        orientation="h",
        y=1.03, x=0,
        bgcolor="rgba(255,255,255,0.85)",
        bordercolor="lightgray",
        borderwidth=1,
        font=dict(family=font_family)
    ),
    updatemenus=[dict(
        type="dropdown",
        showactive=True,
        x=1.1, y=1.25,
        buttons=buttons,
        font=dict(family=font_family, size=13)
    )]
)

fig.update_xaxes(
    title=dict(text="Mother's Qualification Rank", font=dict(family=font_family, size=16)),
    tickfont=dict(family=font_family, size=12)
)
fig.update_yaxes(
    title=dict(text="Father's Qualification Rank", font=dict(family=font_family, size=16)),
    tickfont=dict(family=font_family, size=12)
)

# ------------------------------------------------------
# Save File
# ------------------------------------------------------
pio.write_html(
    fig,
    file="viz1_ses.html",
    include_plotlyjs="cdn",
    full_html=True
)

print("Saved: viz1_ses.html")



Saved: viz1_ses.html


In [7]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio

# Load data 
grade_col = "Curricular units 1st sem (grade)"
units_col = "Curricular units 1st sem (approved)"

df_acad = df.dropna(subset=[grade_col, units_col, "Target"]).copy()

# Font family used across all other visualizations
font_family = "Georgia, 'Times New Roman', serif"

# Thresholds
grade_thr = df_acad[grade_col].median()
units_thr = df_acad[units_col].median()

# Hover variables
hover_cols = []
for col in ["Tuition fees up to date", "Debtor", "Scholarship holder"]:
    if col in df_acad.columns:
        hover_cols.append(col)

# Figure
fig_risk = go.Figure()

outcomes_order = ["Dropout", "Graduate", "Enrolled"]
outcome_colors = {"Dropout": "red", "Graduate": "green", "Enrolled": "orange"}

scatter_indices = {}
line_indices = {}

# ---------------------------------------------------
# Scatter Plots and Trend Line
# ---------------------------------------------------
for outcome in outcomes_order:
    color = outcome_colors[outcome]
    sub = df_acad[df_acad["Target"] == outcome].copy()

    hover_text = []
    for _, row in sub.iterrows():
        parts = [
            f"Outcome: {outcome}",
            f"Grade (1st Sem): {row[grade_col]:.2f}",
            f"Units Approved: {row[units_col]}",
        ]
        for c in hover_cols:
            parts.append(f"{c}: {row[c]}")
        hover_text.append("<br>".join(parts))

    # Scatter
    scatter_trace = go.Scatter(
        x=sub[grade_col],
        y=sub[units_col],
        mode="markers",
        name=outcome,
        marker=dict(color=color, size=9, opacity=0.75, line=dict(width=0.4, color="black")),
        text=hover_text,
        hovertemplate="%{text}<extra></extra>",
        visible=(outcome == "Dropout")
    )
    fig_risk.add_trace(scatter_trace)
    scatter_indices[outcome] = len(fig_risk.data) - 1

    # Trend line
    if len(sub) >= 2:
        sub_sorted = sub.sort_values(grade_col)
        x_vals = sub_sorted[grade_col].values
        y_vals = sub_sorted[units_col].values

        m, b = np.polyfit(x_vals, y_vals, 1)
        x_line = np.linspace(x_vals.min(), x_vals.max(), 100)
        y_line = m * x_line + b

        line_trace = go.Scatter(
            x=x_line,
            y=y_line,
            mode="lines",
            name=f"{outcome} trend",
            line=dict(color=color, width=2),
            showlegend=False,
            visible=(outcome == "Dropout"),
        )
        fig_risk.add_trace(line_trace)
        line_indices[outcome] = len(fig_risk.data) - 1
    else:
        line_indices[outcome] = None

# ---------------------------------------------------
# Quadrant Lines
# ---------------------------------------------------
fig_risk.add_shape(
    type="line",
    x0=grade_thr, x1=grade_thr,
    y0=df_acad[units_col].min(), y1=df_acad[units_col].max(),
    line=dict(color="grey", width=2, dash="dash")
)

fig_risk.add_shape(
    type="line",
    x0=df_acad[grade_col].min(), x1=df_acad[grade_col].max(),
    y0=units_thr, y1=units_thr,
    line=dict(color="grey", width=2, dash="dash")
)

# ---------------------------------------------------
# Annotations
# ---------------------------------------------------
x_min, x_max = df_acad[grade_col].min(), df_acad[grade_col].max()
y_min, y_max = df_acad[units_col].min(), df_acad[units_col].max()

fig_risk.add_annotation(
    x=(x_min + grade_thr) / 2,
    y=(y_min + units_thr) / 2,
    text="Low grades<br>Low approvals<br><b>Higher dropout risk</b>",
    showarrow=False,
    font=dict(size=12, color="firebrick", family=font_family),
    bgcolor="rgba(255,230,230,0.85)"
)

fig_risk.add_annotation(
    x=(x_max + grade_thr) / 2,
    y=(y_max + units_thr) / 2,
    text="High grades<br>High approvals<br><b>Lower dropout risk</b>",
    showarrow=False,
    font=dict(size=12, color="darkgreen", family=font_family),
    bgcolor="rgba(230,255,230,0.85)"
)

# ---------------------------------------------------
# Dropdown Filter
# ---------------------------------------------------
total_traces = len(fig_risk.data)
buttons = []

# All outcomes button
buttons.append(dict(
    label="All outcomes",
    method="update",
    args=[
        {"visible": [True] * total_traces},
        {"title": {"text": "<b>Academic Risk Map (All Outcomes)</b>"}}
    ]
))

# Single-outcome filters
for outcome in outcomes_order:
    visible = [False] * total_traces
    visible[scatter_indices[outcome]] = True
    if line_indices[outcome] is not None:
        visible[line_indices[outcome]] = True

    buttons.append(dict(
        label=f"{outcome} only",
        method="update",
        args=[
            {"visible": visible},
            {"title": {"text": f"<b>Academic Risk Map ({outcome} Only)</b>"}}
        ]
    ))

# ---------------------------------------------------
# Layout and Fonts
# ---------------------------------------------------
fig_risk.update_layout(
    title=dict(
        text="<b>Academic Risk Map (Dropout Only)</b>",
        font=dict(family=font_family, size=22, color="#2C5F8D")
    ),
    font=dict(family=font_family, size=14, color="#333"),
    hoverlabel=dict(font=dict(family=font_family, size=13)),
    width=1100,
    height=650,
    margin=dict(l=60, r=40, t=100, b=60),

    legend=dict(
        title="Student Outcome",
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="left",
        x=0,
        bgcolor="rgba(255,255,255,0.85)",
        bordercolor="lightgray",
        borderwidth=1,
        font=dict(family=font_family)
    ),

    updatemenus=[
        dict(
            type="dropdown",
            showactive=True,
            x=1.12,
            y=1.15,
            buttons=buttons,
            font=dict(family=font_family, size=13)
        )
    ]
)

fig_risk.update_xaxes(
    title=dict(text="<b>1st Semester Average Grade</b>", font=dict(family=font_family, size=16)),
    tickfont=dict(family=font_family, size=12)
)

fig_risk.update_yaxes(
    title=dict(text="<b>Units Approved in 1st Semester</b>", font=dict(family=font_family, size=16)),
    tickfont=dict(family=font_family, size=12)
)

# ---------------------------------------------------
# Save HTML
# ---------------------------------------------------
pio.write_html(
    fig_risk,
    file="viz2_academic_risk.html",
    include_plotlyjs="cdn",
    full_html=True,
    auto_open=False
)

print("Saved, viz2_academic_risk.html")

Saved, viz2_academic_risk.html
